In [ ]:
!git clone https://github.com/leonardozilli/FrameTrigger.git
%cd FrameTrigger/

In [1]:
!pip install transformers[torch] datasets -q

In [ ]:
from src.fn17 import load_dataset_hf, load_dataset_nltk
from src.train import train, test
from src.predict import predict
from src.process_data import prepare_data

In [2]:
CHECKPOINT = 'bert-base-cased'
TASK = 'frames'

In [ ]:
dataset = load_dataset_nltk()
tokenized_dataset = prepare_data(dataset, CHECKPOINT, task=TASK)
tokenized_dataset

## Train

In [18]:
OUT_DIR = './models'
N_EPOCHS = 10
BATCH_SIZE = 64
LEARNING_RATE = 2e-5

In [19]:
train(pretrained_model=CHECKPOINT, task=TASK, dataset=tokenized_dataset,
    epochs=N_EPOCHS, batch_size=BATCH_SIZE, lr=LEARNING_RATE, model_output_path=OUT_DIR)

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


  0%|          | 0/540 [00:00<?, ?it/s]

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


KeyboardInterrupt: 

## Test

In [ ]:
test(OUT_DIR, tokenized_dataset['test'], task=TASK)

## Predict

In [74]:
sentence = "When the moon hits your eye, that's 'amore'."
sentence2 = "The hallway smelt of boiled cabbage and old rag mats."

predict(pretrained_model=OUT_DIR, sentence=sentence, task=TASK, visualize=True)

"When* the moon hits* your eye* , that ' s ' amore ' ."